# Ultrack I2K 2023 - Introduction

This tutorial will introduce the basic concepts of Ultrack and how to use it to track cells when segmentation is already available.

## Setting up Colab runtime

If you are using Colab, we recommend to set up the runtime to use a GPU.
To do so, go to `Runtime > Change runtime type` and select `GPU` as the hardware accelerator.

## Setup Dependencies

This step is only necessary if you are on Colab or don't have the required packages.

IMPORTANT: The runtime must be initialized.

Uncomment and run the following commands to install all required packages.

In [ ]:
# !pip install stackview cellpose 'napari[all]' ultrack ipycanvas==0.11 cucim 
# !pip install git+https://github.com/Janelia-Trackathon-2023/traccuracy

## Download Dataset

Download the Fluo-C2DL-Huh7 dataset from the [Cell Tracking Challenge](celltrackingchallenge.net), which contains fluorescence microscopy images for cell tracking.

The dataset will be used for demonstrating the segmentation and tracking workflow.

In [ ]:
!wget -nc http://data.celltrackingchallenge.net/training-datasets/Fluo-C2DL-Huh7.zip
!unzip -n Fluo-C2DL-Huh7.zip

## Import Libraries

Import the libraries needed for reading images, processing them, cell segmentation, tracking, and performance metrics. 

In [ ]:
from pathlib import Path
from cellpose.models import Cellpose

import numpy as np
import stackview
from dask.array.image import imread
from numpy.typing import ArrayLike
from rich import print

from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatched
from traccuracy.metrics import CTCMetrics

from ultrack import track, to_tracks_layer, tracks_to_zarr, to_ctc
from ultrack.utils import labels_to_edges
from ultrack.config import MainConfig
from ultrack.imgproc import normalize
from ultrack.imgproc.segmentation import Cellpose
from ultrack.utils.array import array_apply

## Colab or Local

Change the `COLAB` variable to `True` or `False` depending on whether you are running this notebook on Colab or locally.

When running locally napari will be used a the image viewer, while on Colab the images will be displayed using `stackview`.

In [ ]:
COLAB = True
# COLAB = False

if COLAB:
    viewer = None

    # fixes colab encoding error
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"

    # enabling colab output
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except ModuleNotFoundError as e:
        print(e)
else:
    import napari
    from napari.utils import nbscreenshot

    viewer = napari.Viewer()

    def screenshot() -> None:
        display(nbscreenshot(viewer))

## Load Data

Load the Fluo-C2DL-Huh7 dataset.

In [ ]:
dataset = "02"
path = Path("Fluo-C2DL-Huh7") / dataset
image = imread(str(path / "*.tif"))

if COLAB:
    display(stackview.slice(image))
else:
    viewer.add_image(image)
    screenshot()

## Cellpose Segmentation

Use the Cellpose model to segment cells within each frame. The function predict applies Cellpose segmentation to each frame after normalizing it.

In [ ]:
cellpose = Cellpose(model_type="cyto2", gpu=True)

def predict(frame: ArrayLike, gamma: float) -> ArrayLike:
    norm_frame = normalize(np.asarray(frame), gamma=gamma)
    return cellpose(norm_frame, tile=False, normalize=False, diameter=75.0)

cellpose_labels = np.zeros(image.shape, dtype=np.int32)
array_apply(
    image,
    out_array=cellpose_labels,
    func=predict,
    gamma=0.5,
)

## View Segmentations

After obtaining segmented labels, visualize them alongside the original images. This helps to inspect the quality of the segmentation.

In [ ]:
if COLAB:
    display(stackview.curtain(image, cellpose_labels))
else:
    layer = viewer.add_labels(cellpose_labels)
    screenshot()
    layer.visible = False

## Extract Contours and Detection

We converted the segmentation labels to contour and detection maps.
These maps are the intermediate representation of used by Ultrack.

In [ ]:
detection, contours = labels_to_edges(cellpose_labels, sigma=5.0)

In [ ]:
if COLAB:
    display(stackview.curtain(image, detection))
else:
    layer = viewer.add_labels(detection)
    screenshot()
    layer.visible = False

In [ ]:
if COLAB:
    display(stackview.curtain(image, contours))
else:
    layer = viewer.add_image(contours)
    screenshot()
    layer.visible = False

## Configuration

Set tracking parameters with ultrack's `MainConfig`.

The `track` procedure is composed of three steps that can also be called individually:
- `segment`: Computes the segmentation hypotheses for tracking;
- `link`: Links and assign edge weights to the segmentation hypotheses;
- `solve`: Solves the tracking problem by selecting the strongly connected segmentation hypotheses.

Each of these steps requires its own configuration, which we'll set up below. Its documentation can be found [here](https://github.com/royerlab/ultrack/blob/main/ultrack/config/README.md).

In [ ]:
config = MainConfig()

# Candidate segmentation parameters
config.segmentation_config.n_workers = 4
config.segmentation_config.min_area = 2500
config.segmentation_config.min_frontier = 0.1

# Setting the maximum number of candidate neighbors and maximum spatial distance between cells
config.linking_config.max_neighbors = 5
config.linking_config.max_distance = 100
config.linking_config.n_workers = 4

# Adding absurd weight to division because there are few dividing cells
config.tracking_config.division_weight = -100

# Very few tracks enter/leave the field of view, increasing penalization
config.tracking_config.disappear_weight = -1
config.tracking_config.appear_weight = -1

print(config)

## Tracking

Run the tracking algorithm based on the provided configuration, detected regions, and contours.

In [ ]:
track(
    config,
    detection=detection,
    edges=contours,
    overwrite=True
)

## Exporting and Visualization

The intermediate tracking data is stored on disk and must be exported to your preferred format.
Here, we convert the resulting tracks to a DataFrame and Zarr to visualize using napari if running locally.

In [ ]:
tracks_df, graph = to_tracks_layer(config)
tracks_df.to_csv(f"{dataset}_tracks.csv", index=False)

segments = tracks_to_zarr(
    config,
    tracks_df,
    overwrite=True,
)

if COLAB:
    display(stackview.curtain(image, segments))
else:
    viewer.add_tracks(
        tracks_df[["track_id", "t", "y", "x"]],
        name="tracks",
        graph=graph,
        visible=True,
    )

    viewer.add_labels(segments, name="segments").contour = 2
    screenshot()

## Run Metrics

Finally, we evaluate the tracking performance using `traccuracy` with the metrics and annotations from the [Cell Tracking Challenge](celltrackingchallenge.net).

In [ ]:
name = f"{path.parent.name}_{path.name}".upper()
output_path = Path(name) / "TRA"
to_ctc(output_path, config, overwrite=True)

gt_path = path.parent / f"{dataset}_GT" / "TRA"

run_metrics(
    gt_data=load_ctc_data(gt_path),
    pred_data=load_ctc_data(output_path),
    matcher=CTCMatched,
    metrics=[CTCMetrics],
)["CTCMetrics"]